## 『本次練習內容』
#### 使用Xception backbone做 Trnasfer Learning


## 『本次練習目的』
  #### 了解如何使用Transfer Learning
  #### 了解Transfer Learning的優點，可以觀察模型收斂速度

##### 可以自行嘗試多種架構

In [1]:
from tensorflow import keras
from tensorflow.keras.layers import Input


input_tensor = Input(shape=(32, 32, 3))
#include top 決定要不要加入 fully Connected Layer

'''Xception 架構'''
model=keras.applications.xception.Xception(include_top=False, weights='imagenet',
                                           input_tensor=input_tensor,pooling=None, classes=2)

'''Resnet 50 架構'''
#model=keras.applications.ResNet50(include_top=False, weights='imagenet',
                                    #input_tensor=input_tensor,
                                    #pooling=None, classes=10)
model.summary()


Model: "xception"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 15, 15, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 15, 15, 32)   128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 15, 15, 32)   0           block1_conv1_bn[0][0]            
___________________________________________________________________________________________

## 添加層數

In [2]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import GlobalAveragePooling2D

x = model.output

'''可以參考Cifar10實作章節'''
x = GlobalAveragePooling2D()(x)
x = Dropout(rate=0.5)(x)
predictions = Dense(units=10,activation='softmax')(x)

model = Model(inputs=model.input, outputs=predictions)
print('Model深度：', len(model.layers))


Model深度： 135


## 鎖定特定幾層不要更新權重

In [3]:
for layer in model.layers[:3]:
    layer.trainable = False
for layer in model.layers[3:]:
    layer.trainable = True

## 準備 Cifar 10 資料

In [4]:
from tensorflow.keras.datasets import cifar10
import numpy as np
from sklearn.preprocessing import OneHotEncoder

(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape) #(50000, 32, 32, 3)

## Normalize Data
def normalize(X_train,X_test):
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7)
        return X_train, X_test
    
    
## Normalize Training and Testset    
x_train_normalize, x_test_normalize = normalize(x_train, x_test) 

## OneHot Label 由(None, 1)-(None, 10)
## ex. label=2,變成[0,0,1,0,0,0,0,0,0,0]
one_hot=OneHotEncoder(categories='auto')
y_train_one_hot=one_hot.fit_transform(y_train).toarray()
y_test_one_hot=one_hot.transform(y_test).toarray()

(50000, 32, 32, 3)


## Training

In [5]:
# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
model.fit(x_train_normalize,y_train_one_hot,batch_size=32,epochs=20)

Train on 50000 samples
Epoch 1/20
50000/50000 [==============================] - 83s 2ms/sample - loss: 1.3688 - accuracy: 0.5230
Epoch 2/20
50000/50000 [==============================] - 69s 1ms/sample - loss: 0.7851 - accuracy: 0.7423
Epoch 3/20
50000/50000 [==============================] - 69s 1ms/sample - loss: 0.6044 - accuracy: 0.8025
Epoch 4/20
50000/50000 [==============================] - 70s 1ms/sample - loss: 0.4823 - accuracy: 0.8432
Epoch 5/20
50000/50000 [==============================] - 69s 1ms/sample - loss: 0.3935 - accuracy: 0.8700
Epoch 6/20
50000/50000 [==============================] - 70s 1ms/sample - loss: 0.3213 - accuracy: 0.8930
Epoch 7/20
50000/50000 [==============================] - 70s 1ms/sample - loss: 0.2651 - accuracy: 0.9121
Epoch 8/20
50000/50000 [==============================] - 70s 1ms/sample - loss: 0.2252 - accuracy: 0.9261
Epoch 9/20
50000/50000 [==============================] - 70s 1ms/sample - loss: 0.1799 - accuracy: 0.9394
Epoch 10/20
50

In [6]:
scores=model.evaluate(x_train_normalize,y_train_one_hot,verbose=2)

50000/1 - 20s - loss: 0.5422 - accuracy: 0.8372


In [7]:
scores=model.evaluate(x_test_normalize,y_test_one_hot,verbose=2)

10000/1 - 4s - loss: 1.2858 - accuracy: 0.7097
